# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapa'a,22.0752,-159.3190,294.14,88,0,3.09,US,1700929568
1,1,tucuma,-0.1833,-50.0500,304.54,56,53,4.20,BR,1700929568
2,2,ghat,24.9647,10.1728,292.11,32,0,3.93,LY,1700929568
3,3,margaret river,-33.9500,115.0667,292.25,58,100,7.23,AU,1700929460
4,4,tiksi,71.6872,128.8694,249.92,96,100,1.61,RU,1700929568


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size = "Humidity",
    scale = 0.7,
    color = "City")

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
reduced_city_df = city_data_df.loc[(city_data_df["Max Temp"]>260) & 
                                   (city_data_df["Max Temp"]<280) & 
                                   (city_data_df["Wind Speed"]<4.5) & 
                                   (city_data_df["Cloudiness"]==0)]



# Drop any rows with null values
reduced_city_df = reduced_city_df.dropna()

# Display sample data
reduced_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,cromwell,41.5951,-72.6454,276.45,42,0,2.57,US,1700929570
37,37,bethel,41.3712,-73.4140,276.28,50,0,1.34,US,1700929571
63,63,jamestown,42.0970,-79.2353,275.92,64,0,2.68,US,1700929574
71,71,warren,39.4334,-84.1666,277.25,63,0,4.12,US,1700929575
78,78,hohhot,40.8106,111.6522,271.43,57,0,1.00,CN,1700929577
106,106,newman,37.3138,-121.0208,279.39,70,0,0.89,US,1700929581
139,139,hamilton,39.1834,-84.5333,277.99,63,0,3.09,US,1700929584
315,315,zhangye,38.9342,100.4517,276.50,28,0,3.25,CN,1700929603
369,369,lucas,41.6501,-83.6666,275.44,63,0,2.57,US,1700929608
373,373,wyndmoor,40.0812,-75.1893,277.08,56,0,3.60,US,1700929608


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = reduced_city_df.copy()
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
19,cromwell,US,41.5951,-72.6454,42,
37,bethel,US,41.3712,-73.4140,50,
63,jamestown,US,42.0970,-79.2353,64,
71,warren,US,39.4334,-84.1666,63,
78,hohhot,CN,40.8106,111.6522,57,
106,newman,US,37.3138,-121.0208,70,
139,hamilton,US,39.1834,-84.5333,63,
315,zhangye,CN,38.9342,100.4517,28,
369,lucas,US,41.6501,-83.6666,63,
373,wyndmoor,US,40.0812,-75.1893,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

print("Search Complete")

# Display sample data
hotel_df

Starting hotel search
cromwell - nearest hotel: SpringHill Suites Hartford Cromwell
bethel - nearest hotel: Hampton Inn Danbury
jamestown - nearest hotel: DoubleTree Jamestown
warren - nearest hotel: No hotel found
hohhot - nearest hotel: Yue Jia Budget Hotel (foreigners approved)
newman - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
lucas - nearest hotel: Hawthorn Suites by Wyndham Holland/Toledo Area
wyndmoor - nearest hotel: Hilton Garden Inn Philadelphia/Fort Washington
shingu - nearest hotel: ホテルニューパレス
aral - nearest hotel: No hotel found
york - nearest hotel: Wingate
westport - nearest hotel: Norwalk Inn and Conference Center
akureyri - nearest hotel: Hótel Norðurland
qamdo - nearest hotel: 银杏商务宾馆
mujiayingzi - nearest hotel: No hotel found
goseong - nearest hotel: 프린스호텔
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
stuarts draft - nearest hotel: Days Inn by Wyndham Staunton
Search Complete


,City,Country,Lat,Lng,Humidity,Hotel Name
19,cromwell,US,41.5951,-72.6454,42,SpringHill Suites Hartford Cromwell
37,bethel,US,41.3712,-73.4140,50,Hampton Inn Danbury
63,jamestown,US,42.0970,-79.2353,64,DoubleTree Jamestown
71,warren,US,39.4334,-84.1666,63,No hotel found
78,hohhot,CN,40.8106,111.6522,57,Yue Jia Budget Hotel (foreigners approved)
106,newman,US,37.3138,-121.0208,70,No hotel found
139,hamilton,US,39.1834,-84.5333,63,North Vista Manor
315,zhangye,CN,38.9342,100.4517,28,Zhangye Hotel 张掖饭店
369,lucas,US,41.6501,-83.6666,63,Hawthorn Suites by Wyndham Holland/Toledo Area
373,wyndmoor,US,40.0812,-75.1893,56,Hilton Garden Inn Philadelphia/Fort Washington


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)